# **SmartBin:** *Versió 2.1.*

> **Aquest notebook ha estat generat amb assistència d’intel·ligència artificial, però totes les decisions sobre les tècniques i metodologies utilitzades han estat preses per una persona humana.**

## 🧩 Descripció general

Aquest notebook forma part del projecte **SmartBin**, un sistema de classificació automàtica de residus mitjançant **visió per computador** i **aprenentatge automàtic**.  
L’objectiu principal d’aquesta versió (2.1) és **optimitzar el model de classificació desenvolupat en la versió anterior (2.0)**, millorant-ne la precisió i reduint el temps d’entrenament mitjançant dues tècniques complementàries d’optimització:

1. **Grid Search:** una aproximació inicial, simple i exhaustiva que prova totes les combinacions possibles dins d’un conjunt predefinit d’hiperparàmetres.  
2. **Optuna:** una optimització adaptativa i intel·ligent que aprèn dels resultats previs i ajusta els paràmetres de manera dinàmica per trobar la millor configuració.

> **Dataset utilitzat:** [Recyclable and Household Waste Classification (Kaggle)](https://www.kaggle.com/datasets/alistairking/recyclable-and-household-waste-classification)

Aquest conjunt de dades conté imatges de diferents categories de residus reciclables i domèstics (com paper, plàstic, metall, vidre o orgànic), organitzades en carpetes segons la seva classe.  
Cada imatge és processada i redimensionada per entrenar un model de visió per computador capaç d’identificar automàticament el tipus de residu corresponent.

---


## Grid Search: una aproximació inicial

El **Grid Search** és un mètode d’optimització **més simple i determinista** que Optuna.  
A diferència d’aquest últim ,que explora l’espai d’hiperparàmetres de manera intel·ligent i dinàmica, el Grid Search **prova totes les combinacions possibles dins d’un conjunt de valors preestablerts**.  
Això permet acotar ràpidament els intervals on és probable que es trobin els millors paràmetres, però sense la complexitat ni l’eficiència adaptativa d’Optuna.

En el codi, aquest procés recorre totes les combinacions possibles de tres hiperparàmetres definits manualment:

- **Velocitat d’aprenentatge (`learning_rate`)**  
- **Mida del lot (`batch_size`)**  
- **Nombre d’èpoques (`num_epochs`)**

Per a cada combinació:
1. Es carreguen les dades d’entrenament i validació amb transformacions d’imatge (redimensionament, conversió a tensor i normalització).  
2. Es crea i entrena una **xarxa neuronal convolucional (CNN)** amb aquests valors.  
3. Es calcula la **pèrdua mitjana de validació (`val_loss`)**, que mesura el rendiment del model.  
4. Es desa el resultat per comparar-lo amb la resta de combinacions.

Quan s’han provat totes les configuracions, el codi **identifica automàticament la combinació amb la pèrdua de validació més baixa**, considerant-la la millor opció dins del conjunt predefinit.  
Per garantir un ús eficient dels recursos, després de cada prova s’allibera la memòria GPU amb `torch.cuda.empty_cache()` i `gc.collect()`.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
import gc  # per netejar la memòria
import time

# Preprocessament
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Hiperparàmetres a provar
learning_rates = [0.0001, 0.0005, 0.001]
batch_sizes = [32, 64]
epoch_options = [5, 10]

# Guardar resultats
best_val_loss = float('inf')
best_config = {}
results = []

# 🔁 Grid Search
for lr in learning_rates:
    for batch_size in batch_sizes:
        for num_epochs in epoch_options:
            print(f"\n🔧 Prova: lr={lr}, batch_size={batch_size}, epochs={num_epochs}")

            # Dataset i Dataloader
            train_dataset = WasteDataset("images", split='train', transform=transform)
            val_dataset = WasteDataset("images", split='val', transform=transform)
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

            # Model
            model = CNN(num_classes=len(train_dataset.classes)).to('cuda')
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            # Entrenament
            for epoch in range(num_epochs):
                model.train()
                for images, labels in train_loader:
                    images, labels = images.to('cuda'), labels.to('cuda')
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

            # Validació
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to('cuda'), labels.to('cuda')
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item() * images.size(0)

            val_loss /= len(val_dataset)
            print(f"📉 Val loss: {val_loss:.4f}")

            results.append((lr, batch_size, num_epochs, val_loss))

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_config = {
                    'learning_rate': lr,
                    'batch_size': batch_size,
                    'num_epochs': num_epochs,
                    'val_loss': val_loss
                }

            # 🧹 Alliberar memòria
            del model
            torch.cuda.empty_cache()
            gc.collect()

# 🏆 Millor combinació
print("\n✅ Millor combinació trobada:")
print(best_config)

## Optimització amb Optuna

Després d’haver establert uns valors de referència mitjançant el **Grid Search**, aquesta secció aprofundeix en la **optimització automàtica dels hiperparàmetres** amb la llibreria **Optuna**.  
A diferència del Grid Search ,que explora combinacions predefinides de manera exhaustiva però limitada, Optuna utilitza un **enfocament adaptatiu i intel·ligent** que aprèn dels resultats obtinguts en cada *trial* per dirigir la cerca cap a les zones més prometedores de l’espai d’hiperparàmetres.

### Principi de funcionament

Optuna treballa mitjançant un sistema de **proves iteratives (*trials*)**, en què cada trial representa un entrenament complet del model amb una combinació diferent d’hiperparàmetres.  
A través del seu mètode de mostreig basat en el **TPE (*Tree-structured Parzen Estimator*)**, la llibreria ajusta progressivament les seves eleccions segons els resultats previs, concentrant-se en aquells valors que minimitzen la **pèrdua de validació**.

A més, el procés incorpora dos mecanismes essencials per millorar l’eficiència:
- **Early Stopping**: atura l’entrenament si no s’observa millora després d’un cert nombre d’èpoques, evitant càlculs innecessaris.  
- **Pruning**: interromp automàticament els *trials* que no mostren resultats prometedors, permetent centrar els recursos en configuracions més adequades.

### Hiperparàmetres optimitzats

En aquest cas, Optuna explora un espai d’hiperparàmetres centrat al voltant dels valors trobats amb el Grid Search:

| Paràmetre | Rang o valors explorats |
|------------|--------------------------|
| `learning_rate` | 0.0001 – 0.001 (escala logarítmica) |
| `batch_size` | [16, 24, 32, 48, 64] |
| `num_epochs` | 3 – 8 |
| `dropout` | 0.3 – 0.7 |

Cada combinació es prova durant un nombre d’èpoques limitat, amb una paciència màxima de tres iteracions sense millora per evitar sobreentrenament.

### Avaluació i selecció del millor model

Durant la optimització, Optuna:
1. Entrena el model amb la configuració proposada en cada *trial*.  
2. Calcula la **pèrdua de validació (`val_loss`)** com a mètrica objectiu.  
3. Actualitza els seus models interns per ajustar la cerca en els següents *trials*.  
4. Desa automàticament l’estat del **millor model trobat** quan s’obté una millora global.

Quan el procés finalitza (després de 15 *trials* o tres hores màximes d’execució), es recupera el **millor conjunt d’hiperparàmetres** i s’entrena el model definitiu, que posteriorment s’avalua sobre el conjunt de **test** per verificar la seva generalització.

### Resultats i informes generats

En finalitzar la optimització, el notebook genera automàticament:
- **Matriu de confusió (`best_model_confusion_matrix.png`)**, que mostra els encerts i errors per classe.  
- **Informe complet (`optimization_report.json`)** amb hiperparàmetres, mètriques i historial dels *trials*.  
- **Gràfic d’història (`optimization_history.png`)**, que il·lustra l’evolució de la pèrdua de validació al llarg dels *trials*.  
- **Model final optimitzat (`optimized_best_model.pth`)**, llest per a la seva reutilització o integració.



In [4]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os
import random
import gc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tqdm import tqdm
import json
from datetime import datetime

# Dataset Class
class WasteDataset(Dataset):
    def __init__(self, root_dir, split, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.image_paths = []
        self.labels = []
        
        print(f"   🔄 Creando dataset {split}...")
        
        for i, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            for subfolder in ['default', 'real_world']:
                subfolder_dir = os.path.join(class_dir, subfolder)
                if not os.path.exists(subfolder_dir):
                    continue
                    
                image_names = [f for f in os.listdir(subfolder_dir) 
                             if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                random.shuffle(image_names)
                
                if split == 'train':
                    image_names = image_names[:int(0.6 * len(image_names))]
                elif split == 'val':
                    image_names = image_names[int(0.6 * len(image_names)):int(0.8 * len(image_names))]
                else:  # test
                    image_names = image_names[int(0.8 * len(image_names)):]
                
                for image_name in image_names:
                    self.image_paths.append(os.path.join(subfolder_dir, image_name))
                    self.labels.append(i)
        
        print(f"   ✅ Dataset {split} creado: {len(self.image_paths)} imágenes")
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        image = Image.open(image_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# CNN Model
class CNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 56 * 56, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

def evaluate_model(model, test_loader, criterion, device, class_names):
    """Evalúa el modelo y retorna métricas detalladas"""
    model.eval()
    all_preds = []
    all_labels = []
    total_loss = 0.0
    
    print("🔍 Evaluando modelo en dataset de test...")
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluación"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    test_loss = total_loss / len(test_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    
    return {
        'test_loss': test_loss,
        'accuracy': accuracy,
        'predictions': all_preds,
        'true_labels': all_labels,
        'classification_report': classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
    }

def plot_confusion_matrix(y_true, y_pred, class_names, save_path='confusion_matrix.png'):
    """Genera y guarda la matriz de confusión"""
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Matriz de Confusión - Mejor Modelo', fontsize=16)
    plt.ylabel('Etiquetas Verdaderas', fontsize=12)
    plt.xlabel('Predicciones', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"📊 Matriz de confusión guardada en: {save_path}")

def generate_report(study, best_metrics, class_names, save_path='optimization_report.json'):
    """Genera un informe completo de la optimización"""
    report = {
        'optimization_info': {
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'n_trials': len(study.trials),
            'best_trial_number': study.best_trial.number,
            'optimization_direction': 'minimize_validation_loss'
        },
        'best_hyperparameters': study.best_trial.params,
        'best_validation_loss': study.best_trial.value,
        'test_metrics': {
            'test_loss': best_metrics['test_loss'],
            'test_accuracy': best_metrics['accuracy']
        },
        'classification_report': best_metrics['classification_report'],
        'trial_history': [
            {
                'trial': t.number,
                'params': t.params,
                'value': t.value,
                'state': str(t.state)
            } for t in study.trials if t.value is not None
        ]
    }
    
    with open(save_path, 'w') as f:
        json.dump(report, f, indent=2)
    
    print(f"📄 Informe completo guardado en: {save_path}")
    return report

def plot_optimization_history(study, save_path='optimization_history.png'):
    """Genera gráfico de la historia de optimización"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Historia de valores
    trial_numbers = [t.number for t in study.trials if t.value is not None]
    values = [t.value for t in study.trials if t.value is not None]
    
    ax1.plot(trial_numbers, values, 'b-o', markersize=4)
    ax1.axhline(y=study.best_value, color='r', linestyle='--', label=f'Mejor: {study.best_value:.4f}')
    ax1.set_xlabel('Trial')
    ax1.set_ylabel('Validation Loss')
    ax1.set_title('Historia de Optimización')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Distribución de valores
    ax2.hist(values, bins=min(20, len(values)), alpha=0.7, color='skyblue', edgecolor='black')
    ax2.axvline(x=study.best_value, color='r', linestyle='--', label=f'Mejor: {study.best_value:.4f}')
    ax2.set_xlabel('Validation Loss')
    ax2.set_ylabel('Frecuencia')
    ax2.set_title('Distribución de Resultados')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"📈 Gráfico de optimización guardado en: {save_path}")

# Configuración
print("🚀 INICIANDO OPTUNA HYPERPARAMETER TUNING")
print("="*50)

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Initialize datasets
print("📂 Cargando datasets...")
dataset_path = 'images'
train_dataset = WasteDataset(dataset_path, 'train', transform)
val_dataset = WasteDataset(dataset_path, 'val', transform)
test_dataset = WasteDataset(dataset_path, 'test', transform)
print(f"📊 Dataset: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)} | {len(train_dataset.classes)} clases")

# Variables globales para tracking
best_global_loss = float('inf')
best_model_state = None
trial_count = 0

def objective(trial):
    global best_global_loss, best_model_state, trial_count
    trial_count += 1
    
    # Hiperparámetros centrados alrededor de tu mejor combinación
    # Learning rate: centrado en 0.0005 con variación
    lr = trial.suggest_float('learning_rate', 0.0001, 0.001, log=True)
    
    # Batch size: probamos tamaños alrededor de 32
    batch_size = trial.suggest_categorical('batch_size', [16, 24, 32, 48, 64])
    
    # Epochs: mantenemos bajo para evitar overfitting, centrado en 5
    epochs = trial.suggest_int('num_epochs', 3, 8)
    
    # Dropout: experimentamos con diferentes valores
    dropout = trial.suggest_float('dropout', 0.3, 0.7, step=0.1)
    
    print(f"\n🔍 Trial {trial_count}/{15}")
    print(f"   📊 Parámetros: lr={lr:.5f}, bs={batch_size}, epochs={epochs}, dropout={dropout:.1f}")
    print(f"   🎯 Baseline a superar: 0.7404 val_loss")
    
    # Crear DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=False)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=False)
    
    # Model setup con dropout personalizable
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = CNN(len(train_dataset.classes)).to(device)
    
    # Modificar dropout del modelo
    model.dropout = nn.Dropout(dropout)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Training loop
    best_val_loss = float('inf')
    patience_counter = 0
    patience = 3  # Paciencia para early stopping
    
    print(f"   🚀 Iniciando entrenamiento...")
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        train_bar = tqdm(train_loader, desc=f"   Epoch {epoch+1}/{epochs} [Train]", leave=False)
        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            
            # Actualizar barra de progreso
            current_train_loss = train_loss / (train_bar.n + 1)
            current_train_acc = 100. * train_correct / train_total
            train_bar.set_postfix({'Loss': f'{current_train_loss:.4f}', 'Acc': f'{current_train_acc:.1f}%'})
        
        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = 100. * train_correct / train_total
        
        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            val_bar = tqdm(val_loader, desc=f"   Epoch {epoch+1}/{epochs} [Val]  ", leave=False)
            for images, labels in val_bar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                
                # Actualizar barra de progreso
                current_val_loss = val_loss / (val_bar.n + 1)
                current_val_acc = 100. * val_correct / val_total
                val_bar.set_postfix({'Loss': f'{current_val_loss:.4f}', 'Acc': f'{current_val_acc:.1f}%'})
        
        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = 100. * val_correct / val_total
        
        # Mostrar resultados del epoch
        improvement_indicator = ""
        if avg_val_loss < 0.7404:
            improvement_indicator = " 🎉"
        elif avg_val_loss < best_val_loss:
            improvement_indicator = " ⬆️"
        
        print(f"   Epoch {epoch+1}: Train={avg_train_loss:.4f}({train_accuracy:.1f}%) | Val={avg_val_loss:.4f}({val_accuracy:.1f}%){improvement_indicator}")
        
        # Report intermediate result para pruning
        trial.report(avg_val_loss, epoch)
        
        # Prune si no es prometedor
        if trial.should_prune():
            print(f"   ✂️ Trial podado - no es prometedor")
            raise optuna.TrialPruned()
        
        # Update best validation loss
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            
            # Si es el mejor modelo global, guardarlo
            if avg_val_loss < best_global_loss:
                best_global_loss = avg_val_loss
                best_model_state = model.state_dict().copy()
                torch.save(best_model_state, 'best_model_temp.pth')
                print(f"   🏆 ¡NUEVO RÉCORD GLOBAL! Val Loss: {avg_val_loss:.6f}")
        else:
            patience_counter += 1
        
        # Early stopping
        if patience_counter >= patience:
            print(f"   ⏰ Early stopping (sin mejora por {patience} epochs)")
            break
    
    # Calcular mejora respecto al baseline
    baseline_loss = 0.7403552888731162
    if best_val_loss < baseline_loss:
        improvement = ((baseline_loss - best_val_loss) / baseline_loss) * 100
        print(f"   ✅ COMPLETADO - Mejor Val Loss: {best_val_loss:.6f} (Mejora: +{improvement:.2f}%)")
    else:
        decline = ((best_val_loss - baseline_loss) / baseline_loss) * 100
        print(f"   ✅ COMPLETADO - Mejor Val Loss: {best_val_loss:.6f} (Decline: -{decline:.2f}%)")
    
    # Cleanup
    del model, train_loader, val_loader
    torch.cuda.empty_cache()
    gc.collect()
    
    return best_val_loss

# Crear estudio de Optuna
print(f"🎯 Objetivo: Superar baseline de 0.7404 validation loss")
print(f"💻 Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
print(f"⏱️ Tiempo estimado: 2-3 horas para 15 trials")

study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(n_startup_trials=5),  # Más trials iniciales para mejor exploración
    pruner=optuna.pruners.MedianPruner(n_startup_trials=3, n_warmup_steps=2)
)

# Ejecutar optimización
print("🚀 Iniciando optimización...")
try:
    study.optimize(objective, n_trials=15, timeout=10800)  # 15 trials, 3h max
    optimization_completed = True
except KeyboardInterrupt:
    print("⏸️ Interrumpido por usuario")
    optimization_completed = True
except Exception as e:
    print(f"❌ Error durante optimización: {e}")
    optimization_completed = False

# Resultados finales
print("\n" + "="*60)
print("🎉 OPTIMIZACIÓN COMPLETADA")
print("="*60)

if len(study.trials) > 0 and study.best_trial:
    print(f"🏆 MEJOR RESULTADO:")
    print(f"   📊 Validation Loss: {study.best_trial.value:.6f}")
    print(f"   ⚙️ Parámetros: {study.best_trial.params}")
    print(f"   🔢 Trial número: {study.best_trial.number}")
    
    baseline_loss = 0.7403552888731162
    if study.best_trial.value < baseline_loss:
        improvement = ((baseline_loss - study.best_trial.value) / baseline_loss) * 100
        print(f"   📈 ¡MEJORA vs baseline!: +{improvement:.2f}%")
    else:
        decline = ((study.best_trial.value - baseline_loss) / baseline_loss) * 100
        print(f"   📉 Decline vs baseline: -{decline:.2f}%")
    
    print(f"\n📈 Resumen de trials:")
    print(f"   ✅ Trials completados: {len([t for t in study.trials if t.value is not None])}")
    print(f"   ✂️ Trials podados: {len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])}")
    print(f"   ❌ Trials fallidos: {len([t for t in study.trials if t.state == optuna.trial.TrialState.FAIL])}")
    
    # Evaluar el mejor modelo en test set
    if os.path.exists('best_model_temp.pth'):
        print(f"\n🧪 EVALUACIÓN EN TEST SET:")
        print("-" * 40)
        
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        best_model = CNN(len(train_dataset.classes)).to(device)
        best_model.load_state_dict(torch.load('best_model_temp.pth'))
        
        # Aplicar dropout del mejor trial
        if 'dropout' in study.best_trial.params:
            best_model.dropout = nn.Dropout(study.best_trial.params['dropout'])
        
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)
        criterion = nn.CrossEntropyLoss()
        
        test_metrics = evaluate_model(best_model, test_loader, criterion, device, train_dataset.classes)
        
        print(f"📊 Test Loss: {test_metrics['test_loss']:.6f}")
        print(f"🎯 Test Accuracy: {test_metrics['accuracy']*100:.2f}%")
        
        # Generar todos los informes y gráficos
        print(f"\n💾 GENERANDO INFORMES:")
        print("-" * 40)
        
        plot_confusion_matrix(test_metrics['true_labels'], test_metrics['predictions'], 
                            train_dataset.classes, 'best_model_confusion_matrix.png')
        
        generate_report(study, test_metrics, train_dataset.classes, 'optimization_report.json')
        
        plot_optimization_history(study, 'optimization_history.png')
        
        # Guardar modelo final
        torch.save(best_model.state_dict(), "optimized_best_model.pth")
        
        print("✅ Archivos generados:")
        print("   📦 optimized_best_model.pth - Mejor modelo")
        print("   📊 best_model_confusion_matrix.png - Matriz de confusión")
        print("   📄 optimization_report.json - Informe completo")
        print("   📈 optimization_history.png - Historia de optimización")
        
        # Limpiar archivo temporal
        if os.path.exists('best_model_temp.pth'):
            os.remove('best_model_temp.pth')
        
        print(f"\n🎊 ¡PROCESO COMPLETADO EXITOSAMENTE!")
        
        # Mostrar recomendaciones
        print(f"\n💡 RECOMENDACIONES:")
        print("-" * 40)
        if study.best_trial.value < baseline_loss:
            print("✅ ¡Excelente! Se encontró una mejora significativa.")
            print("   Considera ejecutar más trials alrededor de estos parámetros.")
        else:
            print("⚠️ No se superó el baseline actual.")
            print("   Considera probar:")
            print("   - Rangos de learning rate más amplios")
            print("   - Más epochs con regularización")
            print("   - Arquitecturas de modelo diferentes")
    
    else:
        print("⚠️ No se encontró el modelo guardado")
        
else:
    print("❌ No se completaron trials exitosos")
    print("🔧 Verifica:")
    print("   - Ruta del dataset")
    print("   - Memoria GPU disponible")
    print("   - Configuración de hiperparámetros")

print("\n🏁 FIN DEL PROCESO")

[I 2025-08-23 17:18:27,903] A new study created in memory with name: no-name-460ab4f7-605c-40cc-8b92-21d76f8b5910


🚀 INICIANDO OPTUNA HYPERPARAMETER TUNING
📂 Cargando datasets...
   🔄 Creando dataset train...
   ✅ Dataset train creado: 9000 imágenes
   🔄 Creando dataset val...
   ✅ Dataset val creado: 3000 imágenes
   🔄 Creando dataset test...
   ✅ Dataset test creado: 3000 imágenes
📊 Dataset: Train=9000, Val=3000, Test=3000 | 30 clases
🎯 Objetivo: Superar baseline de 0.7404 validation loss
💻 Device: CUDA
⏱️ Tiempo estimado: 2-3 horas para 15 trials
🚀 Iniciando optimización...

🔍 Trial 1/15
   📊 Parámetros: lr=0.00016, bs=16, epochs=5, dropout=0.7
   🎯 Baseline a superar: 0.7404 val_loss
   🚀 Iniciando entrenamiento...


   Epoch 1: Train=3.2926(11.5%) | Val=2.8234(29.6%) ⬆️
   🏆 ¡NUEVO RÉCORD GLOBAL! Val Loss: 2.823425


   Epoch 2: Train=2.7218(25.0%) | Val=2.0948(49.8%) ⬆️
   🏆 ¡NUEVO RÉCORD GLOBAL! Val Loss: 2.094762


   Epoch 3: Train=2.1305(40.5%) | Val=1.5749(62.9%) ⬆️
   🏆 ¡NUEVO RÉCORD GLOBAL! Val Loss: 1.574946


...

   Epoch 2: Train=2.6745(26.7%) | Val=2.2870(41.2%) ⬆️


[I 2025-08-23 18:13:13,441] Trial 14 pruned.                                                                           


   Epoch 3: Train=2.2345(38.8%) | Val=1.8930(49.9%) ⬆️
   ✂️ Trial podado - no es prometedor

🎉 OPTIMIZACIÓN COMPLETADA
🏆 MEJOR RESULTADO:
   📊 Validation Loss: 0.724403
   ⚙️ Parámetros: {'learning_rate': 0.0002482301862794532, 'batch_size': 16, 'num_epochs': 8, 'dropout': 0.5}
   🔢 Trial número: 8
   📈 ¡MEJORA vs baseline!: +2.15%

📈 Resumen de trials:
   ✅ Trials completados: 15
   ✂️ Trials podados: 9
   ❌ Trials fallidos: 0

🧪 EVALUACIÓN EN TEST SET:
----------------------------------------
🔍 Evaluando modelo en dataset de test...


Evaluación: 100%|██████████████████████████████████████████████████████████████████████| 94/94 [00:09<00:00, 10.25it/s]


📊 Test Loss: 0.761791
🎯 Test Accuracy: 82.43%

💾 GENERANDO INFORMES:
----------------------------------------
📊 Matriz de confusión guardada en: best_model_confusion_matrix.png
📄 Informe completo guardado en: optimization_report.json
📈 Gráfico de optimización guardado en: optimization_history.png
✅ Archivos generados:
   📦 optimized_best_model.pth - Mejor modelo
   📊 best_model_confusion_matrix.png - Matriz de confusión
   📄 optimization_report.json - Informe completo
   📈 optimization_history.png - Historia de optimización

🎊 ¡PROCESO COMPLETADO EXITOSAMENTE!

💡 RECOMENDACIONES:
----------------------------------------
✅ ¡Excelente! Se encontró una mejora significativa.
   Considera ejecutar más trials alrededor de estos parámetros.

🏁 FIN DEL PROCESO
